In [1]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [2]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [3]:
CONFIG_FILE_PATH = 'ynet_additional_files/config/sdd_trajnet.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'sdd'

TEST_DATA_PATH = 'ynet_additional_files/data/SDD/test_trajnet.pkl'
TEST_IMAGE_PATH = 'ynet_additional_files/data/SDD/test'  # only needed for YNet, PECNet ignores this value
OBS_LEN = 8  # in timesteps
PRED_LEN = 12  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 1  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8

#### Load config file and print hyperparameters

In [4]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.25,
 'batch_size': 8,
 'viz_epoch': 10,
 'learning_rate': 0.0001,
 'num_epochs': 300,
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [11],
 'temperature': 1.0,
 'segmentation_model_fp': 'ynet_additional_files/segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'use_features_only': False,
 'unfreeze': 150,
 'use_TTST': True,
 'rel_threshold': 0.01,
 'use_CWS': False,
 'CWS_params': 'None'}

#### Load preprocessed Data

In [5]:
df_test = pd.read_pickle(TEST_DATA_PATH)

In [6]:
df_test.head()

,frame,trackId,x,y,sceneId,metaId
0,0,28.0,1539.5,578.0,coupa_0,0
1,12,28.0,1484.5,576.0,coupa_0,0
2,24,28.0,1484.5,576.0,coupa_0,0
3,36,28.0,1459.5,571.0,coupa_0,0
4,48,28.0,1432.5,569.0,coupa_0,0


#### Initiate model and load pretrained weights

In [7]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/root/miniconda3/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/root/miniconda3/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/root/miniconda3/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retr

In [8]:
model.load(f'ynet_additional_files/pretrained_models/{experiment_name}_weights.pt')

<All keys matched successfully>


#### Evaluate model

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
model.evaluate(df_test, params, image_path=TEST_IMAGE_PATH,
               batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=device, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 17/17 [00:00<00:00, 1506.10it/s]
/opt/data/private/Human-Path-Prediction/ynet/utils/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(trajectories), meta, scene_list
Round:   0%|          | 0/1 [00:00<?, ?it/s]


Start testing


RuntimeError: CUDA error: no kernel image is available for execution on the device

: 